In [4]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LeakyReLU, BatchNormalization
from keras import backend
from keras.callbacks import TensorBoard
from keras.optimizers import Adam
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [12]:
# Hyperparams
n_units = 400
layers = 4
n_batch = 4096
n_epochs = 50

In [6]:
df = pd.read_csv('options-df-sigma.csv')
df = df.dropna(axis=0)
df = df.drop(columns=['date', 'exdate', 'impl_volatility', 'volume', 'open_interest'])
df.strike_price = df.strike_price / 1000
call_df = df[df.cp_flag == 'C'].drop(['cp_flag'], axis=1)
put_df = df[df.cp_flag == 'P'].drop(['cp_flag'], axis=1)

In [4]:
call_df.head()

,strike_price,best_bid,best_offer,volume,open_interest,date_ndiff,treasury_rate,closing_price,sigma_20
0,600.0,28.000,29.000,4,8555,47,5.17,624.22,0.007761
10,475.0,152.875,153.875,0,2700,145,5.12,624.22,0.007761
13,600.0,52.375,53.375,0,2155,327,5.05,624.22,0.007761
17,610.0,20.000,20.750,923,8860,47,5.17,624.22,0.007761
18,675.0,34.000,35.000,0,350,691,5.10,624.22,0.007761


In [8]:
call_X_train, call_X_test, call_y_train, call_y_test = train_test_split(call_df.drop(['best_bid', 'best_offer'], axis=1),
                                                                        call_df[['best_bid', 'best_offer']],
                                                                        test_size=0.01, random_state=42)
put_X_train, put_X_test, put_y_train, put_y_test = train_test_split(put_df.drop(['best_bid', 'best_offer'], axis=1),
                                                                    put_df[['best_bid', 'best_offer']],
                                                                    test_size=0.01, random_state=42)

In [9]:
model = Sequential()
model.add(Dense(n_units, input_dim=call_X_train.shape[1]))
model.add(LeakyReLU())

for _ in range(layers - 1):
    model.add(Dense(n_units))
    model.add(BatchNormalization())
    model.add(LeakyReLU())

model.add(Dense(2, activation='relu'))

model.compile(loss='mse', optimizer=Adam())

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 400)               2400      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 400)               160400    
_________________________________________________________________
batch_normalization_1 (Batch (None, 400)               1600      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 400)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 400)               160400    
_________________________________________________________________
batch_normalization_2 (Batch (None, 400)               1600      
__________

In [13]:
history = model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=n_epochs, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)


Train on 5001241 samples, validate on 50518 samples


Epoch 1/50
5001241/5001241 [==============================] - 31s 6us/step - loss: 33942.6637 - val_loss: 354.7733
Epoch 2/50
5001241/5001241 [==============================] - 30s 6us/step - loss: 132.7384 - val_loss: 300.3519
Epoch 3/50
5001241/5001241 [==============================] - 31s 6us/step - loss: 119.2084 - val_loss: 1399.3595
Epoch 4/50
5001241/5001241 [==============================] - 30s 6us/step - loss: 118.4050 - val_loss: 855.5378
Epoch 5/50
5001241/5001241 [==============================] - 31s 6us/step - loss: 118.0120 - val_loss: 4310.1394
Epoch 6/50
5001241/5001241 [==============================] - 31s 6us/step - loss: 111.3818 - val_loss: 2980.0545
Epoch 7/50
5001241/5001241 [==============================] - 30s 6us/step - loss: 108.9900 - val_loss: 137.0270
Epoch 8/50
5001241/5001241 [==============================] - 30s 6us/step - loss: 104.2108 - val_loss: 6908.7640
Epoch 9/50
5001241/5001241 [=======

KeyboardInterrupt: 

In [14]:
model.save('mlp2-call30.h5')

In [15]:
call_y_pred30 = model.predict(call_X_test)

In [16]:
print('equilibrium mse', np.mean(np.square(np.mean(call_y_test.values, axis=1) - np.mean(call_y_pred30, axis=1))))
print('spread mse', np.mean(np.square(np.diff(call_y_test.values, axis=1) - np.diff(call_y_pred30, axis=1))))

equilibrium mse 289.3989112870401
spread mse 9.904612136909982


In [17]:
model.compile(loss='mse', optimizer=Adam(1e-4))
history = model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=n_epochs, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

Train on 5001241 samples, validate on 50518 samples
Epoch 1/50
5001241/5001241 [==============================] - 30s 6us/step - loss: 66.2047 - val_loss: 116.5832
Epoch 2/50
5001241/5001241 [==============================] - 29s 6us/step - loss: 62.6459 - val_loss: 68.8921
Epoch 3/50
5001241/5001241 [==============================] - 29s 6us/step - loss: 61.4737 - val_loss: 35.1672
Epoch 4/50
5001241/5001241 [==============================] - 30s 6us/step - loss: 64.1139 - val_loss: 78.1961
Epoch 5/50
5001241/5001241 [==============================] - 29s 6us/step - loss: 61.4858 - val_loss: 301.6294
Epoch 6/50
5001241/5001241 [==============================] - 29s 6us/step - loss: 60.5258 - val_loss: 27.7076
Epoch 7/50
5001241/5001241 [==============================] - 30s 6us/step - loss: 57.3424 - val_loss: 151.8109
Epoch 8/50
5001241/5001241 [==============================] - 29s 6us/step - loss: 59.9215 - val_loss: 95.0608
Epoch 9/50
5001241/5001241 [=============================

KeyboardInterrupt: 

In [18]:
model.save('mlp2-call40.h5')

In [19]:
call_y_pred40 = model.predict(call_X_test)
print('equilibrium mse', np.mean(np.square(np.mean(call_y_test.values, axis=1) - np.mean(call_y_pred40, axis=1))))
print('spread mse', np.mean(np.square(np.diff(call_y_test.values, axis=1) - np.diff(call_y_pred40, axis=1))))

equilibrium mse 287.78500592993214
spread mse 18.71940528005765


In [20]:
model.compile(loss='mse', optimizer=Adam(1e-5))
history = model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=n_epochs, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

Train on 5001241 samples, validate on 50518 samples
Epoch 1/50
5001241/5001241 [==============================] - 31s 6us/step - loss: 55.7768 - val_loss: 27.2359
Epoch 2/50
5001241/5001241 [==============================] - 30s 6us/step - loss: 58.6699 - val_loss: 27.6299
Epoch 3/50
5001241/5001241 [==============================] - 30s 6us/step - loss: 57.3901 - val_loss: 17.0203
Epoch 4/50
5001241/5001241 [==============================] - 30s 6us/step - loss: 54.7001 - val_loss: 19.4312
Epoch 5/50
5001241/5001241 [==============================] - 30s 6us/step - loss: 56.1742 - val_loss: 17.4106
Epoch 6/50
5001241/5001241 [==============================] - 30s 6us/step - loss: 52.9307 - val_loss: 17.8438
Epoch 7/50
5001241/5001241 [==============================] - 30s 6us/step - loss: 56.4020 - val_loss: 22.2832
Epoch 8/50
5001241/5001241 [==============================] - 30s 6us/step - loss: 59.7244 - val_loss: 17.3589
Epoch 9/50
5001241/5001241 [==============================] 

KeyboardInterrupt: 

In [21]:
model.save('mlp2-call50.h5')
call_y_pred50 = model.predict(call_X_test)
print('equilibrium mse', np.mean(np.square(np.mean(call_y_test.values, axis=1) - np.mean(call_y_pred50, axis=1))))
print('spread mse', np.mean(np.square(np.diff(call_y_test.values, axis=1) - np.diff(call_y_pred50, axis=1))))

equilibrium mse 17.790098248083083
spread mse 7.623696717293795


In [22]:
model.compile(loss='mse', optimizer=Adam(1e-6))
history = model.fit(call_X_train, call_y_train, 
                    batch_size=n_batch, epochs=10, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

Train on 5001241 samples, validate on 50518 samples
Epoch 1/10
5001241/5001241 [==============================] - 31s 6us/step - loss: 56.0327 - val_loss: 16.4086
Epoch 2/10
5001241/5001241 [==============================] - 30s 6us/step - loss: 55.4802 - val_loss: 17.1078
Epoch 3/10
5001241/5001241 [==============================] - 31s 6us/step - loss: 55.0783 - val_loss: 16.4661
Epoch 4/10
5001241/5001241 [==============================] - 31s 6us/step - loss: 54.1477 - val_loss: 17.1634
Epoch 5/10
5001241/5001241 [==============================] - 30s 6us/step - loss: 55.6712 - val_loss: 16.4914
Epoch 6/10
5001241/5001241 [==============================] - 30s 6us/step - loss: 56.4268 - val_loss: 16.1995
Epoch 7/10
5001241/5001241 [==============================] - 30s 6us/step - loss: 57.1177 - val_loss: 16.9857
Epoch 8/10
5001241/5001241 [==============================] - 30s 6us/step - loss: 56.3512 - val_loss: 17.1600
Epoch 9/10
5001241/5001241 [==============================] 

In [24]:
model.save('mlp2-call60.h5')
call_y_pred60 = model.predict(call_X_test)
print('equilibrium mse', np.mean(np.square(np.mean(call_y_test.values, axis=1) - np.mean(call_y_pred60, axis=1))))
print('spread mse', np.mean(np.square(np.diff(call_y_test.values, axis=1) - np.diff(call_y_pred60, axis=1))))

equilibrium mse 15.2128482637227
spread mse 7.5714239975514355
